In [56]:
import pandas as pd 
import datetime as dt
import numpy as np
import keras
import plotly.graph_objects as go

from keras.models import Sequential
import tensorflow as tf
from alphaRNN import *
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.regularizers import l1
from sklearn.metrics import mean_squared_error, r2_score

In [57]:
data = pd.read_csv("hourlySummaries.csv")
data = data.set_index("hour")
data = data[data.columns[1:]]
# data = data.iloc[:1000,]

train_weight = 0.80
split = int(len(data)*train_weight)
df_train = data.iloc[:split].copy()
df_test = data.iloc[split:].copy()

In [58]:
X_train = df_train.iloc[:,list(range(1,32))]
X_test = df_test.iloc[:,list(range(1,32))]

y_train = df_train["aaveHourlyChange"]
y_test = df_test["aaveHourlyChange"]
# y_train = df_train["aaveDirectionOfChange"]
# y_test = df_test["aaveDirectionOfChange"]

In [59]:
x_train_reg = np.array(X_train)
x_test_reg = np.array(X_test)
y_train_reg = np.array(y_train)
y_test_reg = np.array(y_test)

In [60]:
x_train_reg = np.reshape(x_train_reg, (x_train_reg.shape[0], x_train_reg.shape[1], 1))
x_test_reg = np.reshape(x_test_reg, (x_test_reg.shape[0], x_test_reg.shape[1], 1))

In [61]:
# model = keras.Sequential()
# model.add(keras.layers.LSTM(50, activation='relu', input_shape=(x_train_reg.shape[1], 1)))
# model.add(keras.layers.Dense(1))
# model.compile(optimizer='adam', loss='mse')

# model.fit(x_train_reg, y_train_reg, epochs=10, validation_split=0.2, verbose=1)

# y_pred = model.predict(x_test_reg)

# Alpha RNN

In [62]:
def AlphaRNN_(n_units = 10, l1_reg=0, seed=0):
  model = Sequential()
  model.add(AlphaRNN(n_units, activation='tanh', kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed), recurrent_initializer=keras.initializers.orthogonal(seed), kernel_regularizer=l1(l1_reg), input_shape=(x_train_reg.shape[1], x_train_reg.shape[-1]), unroll=True))
  model.add(Dense(1, kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed), kernel_regularizer=l1(l1_reg)))
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

In [63]:
max_epoches=1000
batch_size=1000

es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=100, min_delta=1e-7, restore_best_weights=True)

# params = {'alpharnn': {'model':'', 'function':AlphaRNN_,'l1_reg':0.0, 'H':10,'color': 'green', 'label': '$\\alpha$-RNN' }}

In [64]:
model = AlphaRNN_(10,0.0)
model.fit(x_train_reg,y_train_reg,epochs=max_epoches, batch_size=batch_size,callbacks=[es],shuffle=False)


Epoch 1/1000
8680/8680 [==============================] - 6s 691us/step - loss: 31.8911
Epoch 2/1000
8680/8680 [==============================] - 1s 73us/step - loss: 30.7841
Epoch 3/1000
8680/8680 [==============================] - 1s 75us/step - loss: 29.7502
Epoch 4/1000
8680/8680 [==============================] - 1s 75us/step - loss: 28.8435
Epoch 5/1000
8680/8680 [==============================] - 1s 72us/step - loss: 28.1217
Epoch 6/1000
8680/8680 [==============================] - 1s 73us/step - loss: 27.6143
Epoch 7/1000
8680/8680 [==============================] - 1s 74us/step - loss: 27.3009
Epoch 8/1000
8680/8680 [==============================] - 1s 76us/step - loss: 27.1274
Epoch 9/1000
8680/8680 [==============================] - 1s 73us/step - loss: 27.0366
Epoch 10/1000
8680/8680 [==============================] - 1s 103us/step - loss: 26.9864
Epoch 11/1000
8680/8680 [==============================] - 1s 76us/step - loss: 26.9531
Epoch 12/1000
8680/8680 [==============

In [65]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
alpha_rnn_6 (AlphaRNN)       (None, 10)                121       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 132
Trainable params: 132
Non-trainable params: 0
_________________________________________________________________


In [66]:
y_pred_train = model.predict(x_train_reg, verbose = 1)
MSE_train= mean_squared_error(y_train,y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

print("Train MSE: ", MSE_train)
print("Train r2: ", r2_train)

y_pred = model.predict(x_test_reg, verbose = 1)
MSE_test= mean_squared_error(y_test,y_pred)
r2_test = r2_score(y_test, y_pred)

print("Test MSE: ", MSE_test)
print("Test r2: ", r2_test)

8680/8680 [==============================] - 2s 226us/step
Train MSE:  26.534069527843723
Train r2:  0.0038150527931799516
2171/2171 [==============================] - 0s 103us/step
Test MSE:  4.82057354635082
Test r2:  0.0006715994948730541


In [79]:
import time
pred_prices = [df_test["aavePriceUSD"].iloc[0]]
for i in range(1,len(y_test)):
    pct = y_test[i] /100
    pred_prices.append(pred_prices[-1] * (1 + pct))

In [81]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_test.index, y=pred_prices,
                    mode='lines',
                    name='Aave Predicted'))
fig.add_trace(go.Scatter(x=df_test.index, y=df_test.aavePriceUSD,
                    mode='lines',
                    name='Aave Actual'))

# Using changes in price to predict numBorrows

In [43]:
X_train = df_train["aaveHourlyChange"]
X_test = df_test["aaveHourlyChange"]

y_train = df_train[["numBorrows"]]
y_test = df_test[["numBorrows"]]

In [44]:
x_train_reg = np.array(X_train)
x_test_reg = np.array(X_test)
y_train_reg = np.array(y_train)
y_test_reg = np.array(y_test)

In [45]:
x_train_reg = np.reshape(x_train_reg, (x_train_reg.shape[0], 1, 1))
x_test_reg = np.reshape(x_test_reg, (x_test_reg.shape[0], 1, 1))

In [46]:
def AlphaRNN_(n_units = 10, l1_reg=0, seed=0):
  model = Sequential()
  model.add(AlphaRNN(n_units, activation='tanh', kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed), recurrent_initializer=keras.initializers.orthogonal(seed), kernel_regularizer=l1(l1_reg), input_shape=(x_train_reg.shape[1], x_train_reg.shape[-1]), unroll=True))
  model.add(Dense(1, kernel_initializer=keras.initializers.glorot_uniform(seed), bias_initializer=keras.initializers.glorot_uniform(seed), kernel_regularizer=l1(l1_reg)))
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

In [47]:
model = AlphaRNN_(10,0.0)
model.fit(x_train_reg,y_train_reg,epochs=max_epoches, batch_size=batch_size,callbacks=[es],shuffle=False)

Epoch 1/1000
8680/8680 [==============================] - 1s 98us/step - loss: 1094.1053
Epoch 2/1000
8680/8680 [==============================] - 0s 5us/step - loss: 1090.8272
Epoch 3/1000
8680/8680 [==============================] - 0s 6us/step - loss: 1087.6221
Epoch 4/1000
8680/8680 [==============================] - 0s 6us/step - loss: 1084.4456
Epoch 5/1000
8680/8680 [==============================] - 0s 6us/step - loss: 1081.2859
Epoch 6/1000
8680/8680 [==============================] - 0s 6us/step - loss: 1078.1346
Epoch 7/1000
8680/8680 [==============================] - 0s 6us/step - loss: 1074.9832
Epoch 8/1000
8680/8680 [==============================] - 0s 6us/step - loss: 1071.8247
Epoch 9/1000
8680/8680 [==============================] - 0s 7us/step - loss: 1068.6551
Epoch 10/1000
8680/8680 [==============================] - 0s 9us/step - loss: 1065.4737
Epoch 11/1000
8680/8680 [==============================] - 0s 8us/step - loss: 1062.2789
Epoch 12/1000
8680/8680 [====

In [48]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
alpha_rnn_5 (AlphaRNN)       (None, 10)                121       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 132
Trainable params: 132
Non-trainable params: 0
_________________________________________________________________


In [49]:
y_pred_train = model.predict(x_train_reg, verbose = 1)
MSE_train= mean_squared_error(y_train,y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

print("Train MSE: ", MSE_train)
print("Train r2: ", r2_train)

y_pred = model.predict(x_test_reg, verbose = 1)
MSE_test= mean_squared_error(y_test,y_pred)
r2_test = r2_score(y_test, y_pred)

print("Test MSE: ", MSE_test)
print("Test r2: ", r2_test)

8680/8680 [==============================] - 0s 21us/step
Train MSE:  315.48493410122313
Train r2:  -0.0007072637587020036
2171/2171 [==============================] - 0s 13us/step
Test MSE:  494.0600382006721
Test r2:  -9.228498811360968
